## Some storytelling for a crowdfunding platform

### Background information

A start up has opened a year ago and now they want to take the next step and expand the services. The business model is to run a platform where people who have a business idea, but not the money needed, can sign up and raise money for their project within a given time. On the other hand, there are funders who would like to invest their money in projects. As an intermediary, the platform brings borrowers and lenders together. It earns money with a commission for each project that lands on the platform.

The database is the history of the platform and consists of 2 parts that belong together. All projects are completed projects, i.e. the time to collect money for each of them has expired. The business model is to pay out the collected money even if the target amount was not reached. 

There are no duplicates in the data set.


The split record contains the following columns (including description):

    - funded_amount ... amount received at the end of the project collection period in USD
    - loan_amount ... target amount (amount necessary for the project) in USD
    - activity ... subcategory to which the goal of the project belongs thematically
    - sector ... upper category to which the project belongs thematically
    - use ... short description of what the money will be used for
    - country_code ... country code according to ISO 3166 alpha-2 standard
    - country ... country name according to ISO standard
    - region ... region
    - currency ... currency in which the funded_amount was then paid out
    - term in months ... duration over which the loan was disbursed
    - lender_count ... lender (i.e. how many people have given money for the project)
    - borrower_genders ... gender and number of borrowers, i.e. those who initiated the project
    - repayment interval ... repayment modalities/rhythm
    

#### The main question is: based on the facts represented in the dataset, where could opportunities for growth be found? Where can the company win or save money?

#### Installing needed software

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [2]:
# Installing a convertor which will later generate ISO 3166-1 alpha-3 country codes.

# Choropleth maps can work with country names (available in the dataset)
# but in case of several dozens of countries the probability is high that some names won't be recognized by Plotly,
# therefore I want to create ISO 3166 alpha-3 country codes based on existing alpha-2 country codes 
# - this should help reach unambiguity.

!pip install pycountry-convert

from pycountry_convert import country_alpha2_to_country_name, country_name_to_country_alpha3

### Data preprocessing

#### Examining the dataset

In [3]:
# Reading the data
df1 = pd.read_csv("part1.csv", index_col=0)

In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 335000 entries, 0 to 334999
Data columns (total 13 columns):
funded_amount         335000 non-null float64
loan_amount           335000 non-null float64
activity              335000 non-null object
sector                335000 non-null object
use                   331291 non-null object
country_code          334998 non-null object
country               335000 non-null object
region                296293 non-null object
currency              335000 non-null object
term_in_months        335000 non-null float64
lender_count          335000 non-null int64
borrower_genders      331299 non-null object
repayment_interval    335000 non-null object
dtypes: float64(3), int64(1), object(9)
memory usage: 35.8+ MB


In [5]:
df2 = pd.read_csv("part2.csv", sep="#", index_col=0) # Specifying the separator - another one than in the first part

In [6]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 336205 entries, 0 to 336204
Data columns (total 13 columns):
funded_amount         336205 non-null float64
loan_amount           336205 non-null float64
activity              336205 non-null object
sector                336205 non-null object
use                   335682 non-null object
country_code          336199 non-null object
country               336205 non-null object
region                318112 non-null object
currency              336205 non-null object
term_in_months        336205 non-null float64
lender_count          336205 non-null int64
borrower_genders      335685 non-null object
repayment_interval    336205 non-null object
dtypes: float64(3), int64(1), object(9)
memory usage: 35.9+ MB


In [7]:
# The datasets have the same structure. Concatenating them:
df = pd.concat([df1, df2],ignore_index=True)

In [8]:
# examining the resulting dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671205 entries, 0 to 671204
Data columns (total 13 columns):
funded_amount         671205 non-null float64
loan_amount           671205 non-null float64
activity              671205 non-null object
sector                671205 non-null object
use                   666973 non-null object
country_code          671197 non-null object
country               671205 non-null object
region                614405 non-null object
currency              671205 non-null object
term_in_months        671205 non-null float64
lender_count          671205 non-null int64
borrower_genders      666984 non-null object
repayment_interval    671205 non-null object
dtypes: float64(3), int64(1), object(9)
memory usage: 66.6+ MB


There are some missing values in "use", "country_code", "region", and "borrower_genders".

In [9]:
df.describe()

,funded_amount,loan_amount,term_in_months,lender_count
count,671205.000000,671205.000000,671205.000000,671205.000000
mean,785.995061,842.397107,13.739022,20.590922
std,1130.398941,1198.660073,8.598919,28.459551
min,0.000000,25.000000,1.000000,0.000000
25%,250.000000,275.000000,8.000000,7.000000
50%,450.000000,500.000000,13.000000,13.000000
75%,900.000000,1000.000000,14.000000,24.000000
max,100000.000000,100000.000000,158.000000,2986.000000


#### Dealing with missing values

Missings in 'use', 'borrower_genders', and 'region' - ignore them for now, probably I won't use these columns.

Missings in 'country_code' - will be dealt with now: I want to impute values.

In [10]:
df.loc[df['country_code'].isnull()]

,funded_amount,loan_amount,activity,sector,use,country_code,country,region,currency,term_in_months,lender_count,borrower_genders,repayment_interval
202537,4150.0,4150.0,Wholesale,Wholesale,To purchase lighting products for sale to loca...,NaN,Namibia,EEnhana,NAD,6.0,162,female,bullet
202823,4150.0,4150.0,Wholesale,Wholesale,To purchase lighting products for sale to loca...,NaN,Namibia,Rundu,NAD,6.0,159,male,bullet
344929,3325.0,3325.0,Wholesale,Wholesale,To purchase lighting products for sale to loca...,NaN,Namibia,EEnhana,NAD,7.0,120,female,bullet
351177,3325.0,3325.0,Wholesale,Wholesale,To purchase lighting products for sale to loca...,NaN,Namibia,Rundu,NAD,7.0,126,male,bullet
420953,3325.0,3325.0,Wholesale,Wholesale,To purchase lighting products for sale to loca...,NaN,Namibia,EEnhana,NAD,7.0,118,female,bullet
421218,4000.0,4000.0,Wholesale,Wholesale,purchase solar lighting products for sale to l...,NaN,Namibia,Rundu,NAD,7.0,150,male,bullet
487207,5100.0,5100.0,Renewable Energy Products,Retail,to pay for stock of solar lights and cell phon...,NaN,Namibia,Katima Mulilo,NAD,7.0,183,male,bullet
487653,5000.0,5000.0,Wholesale,Wholesale,to maintain a stock of solar lights and cell p...,NaN,Namibia,Oshakati,NAD,7.0,183,female,bullet


In [11]:
# They are all from Namibia, its code is NA (according to ISO 3166 ALPHA-2), so now I will impute this code instead ot NaNs:
df.loc[df['country']=='Namibia', ['country_code']] = 'NA'

In [12]:
# Checking if the codes were correctly assigned:
df.loc[df['country']=='Namibia']
# Yes, they were.

,funded_amount,loan_amount,activity,sector,use,country_code,country,region,currency,term_in_months,lender_count,borrower_genders,repayment_interval
202537,4150.0,4150.0,Wholesale,Wholesale,To purchase lighting products for sale to loca...,NA,Namibia,EEnhana,NAD,6.0,162,female,bullet
202823,4150.0,4150.0,Wholesale,Wholesale,To purchase lighting products for sale to loca...,NA,Namibia,Rundu,NAD,6.0,159,male,bullet
344929,3325.0,3325.0,Wholesale,Wholesale,To purchase lighting products for sale to loca...,NA,Namibia,EEnhana,NAD,7.0,120,female,bullet
351177,3325.0,3325.0,Wholesale,Wholesale,To purchase lighting products for sale to loca...,NA,Namibia,Rundu,NAD,7.0,126,male,bullet
420953,3325.0,3325.0,Wholesale,Wholesale,To purchase lighting products for sale to loca...,NA,Namibia,EEnhana,NAD,7.0,118,female,bullet
421218,4000.0,4000.0,Wholesale,Wholesale,purchase solar lighting products for sale to l...,NA,Namibia,Rundu,NAD,7.0,150,male,bullet
487207,5100.0,5100.0,Renewable Energy Products,Retail,to pay for stock of solar lights and cell phon...,NA,Namibia,Katima Mulilo,NAD,7.0,183,male,bullet
487653,5000.0,5000.0,Wholesale,Wholesale,to maintain a stock of solar lights and cell p...,NA,Namibia,Oshakati,NAD,7.0,183,female,bullet


#### Feature engineering

To use choroplet maps, I'll need country codes in ISO alpha-3 format. For this, I will use a convertor.

In [13]:
# Country_code 'XK' for Kosovo will not be accepted by the ISO alpha-2 to ISO alpha-3 convertor and by choroplet. 
# I am changing it to RS (Serbia) - sorry, Kosovar.

df.loc[df['country_code'] == 'XK', ['country_code']] = 'RS'

In [14]:
# Applying the convertor and adding the column 'country_alpha_3':
df['country_alpha_3'] = df.country_code.apply(lambda x: country_name_to_country_alpha3(country_alpha2_to_country_name(x)))

Now, I create a separate variable measuring success of the crowdfunding for each separate project. 

It will be calculated as follows: (funded_amount / loan_amount) * 100%...

...and then grouped into 4 categories: <br>
- unsuccessful = 0-10% money gathered, <br> 
- partially successfull - more than 10% but equal or less than 50% from what was needed, <br> 
- mostly successful - more than 50% but equal or less than 90%, <br> 
- successful - more than 90% was gathered.

In [15]:
# First, calculating %% of how much money were gathered:
df.loc[:, 'share_gathered'] = df.loc[:, 'funded_amount'] / df.loc[:, 'loan_amount'] * 100

In [16]:
# Second, grouping the variable and creating another variable 'success' with text labels:

categories = ['unsuccessful','partly successful','mostly successful','successful']

min_value = df['share_gathered'].min()
max_value = df['share_gathered'].max()
distribution = [min_value, 10, 50, 90, max_value]

df['success'] = pd.cut(df['share_gathered'], include_lowest=True, bins=distribution, labels=categories)

In [17]:
df.success.unique() # it has worked

[successful, mostly successful, partly successful, unsuccessful]
Categories (4, object): [unsuccessful < partly successful < mostly successful < successful]

#### Optimizing memory usage

Reducing the size of the dataset.

In [18]:
df.info() # initial memory usage: 77.5+ MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671205 entries, 0 to 671204
Data columns (total 16 columns):
funded_amount         671205 non-null float64
loan_amount           671205 non-null float64
activity              671205 non-null object
sector                671205 non-null object
use                   666973 non-null object
country_code          671205 non-null object
country               671205 non-null object
region                614405 non-null object
currency              671205 non-null object
term_in_months        671205 non-null float64
lender_count          671205 non-null int64
borrower_genders      666984 non-null object
repayment_interval    671205 non-null object
country_alpha_3       671205 non-null object
share_gathered        671205 non-null float64
success               671205 non-null category
dtypes: category(1), float64(4), int64(1), object(10)
memory usage: 77.5+ MB


In [19]:
mem1 = df.memory_usage(deep=True).sum()

In [20]:
# Dropping some columns which I believe won't be needed.
df.drop(columns=['region', 'currency', 'borrower_genders'], inplace=True)

In [21]:
# Re-formatting some columns to "category"
df.loc[:, ['activity', 'sector', 'country_code', 'country', 'repayment_interval', 'country_alpha_3']] = df.loc[:, ['activity', 'sector', 'country_code', 'country', 'repayment_interval', 'country_alpha_3']].astype('category')

In [22]:
# Converting all floats to integers 
df.loc[:, ["funded_amount","loan_amount","term_in_months", "lender_count"]] = df.loc[:, ["funded_amount","loan_amount","term_in_months", "lender_count"]].apply(pd.to_numeric, downcast="integer")

In [23]:
df.info() # memory usage after optimization: 23.1+ MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671205 entries, 0 to 671204
Data columns (total 13 columns):
funded_amount         671205 non-null int32
loan_amount           671205 non-null int32
activity              671205 non-null category
sector                671205 non-null category
use                   666973 non-null object
country_code          671205 non-null category
country               671205 non-null category
term_in_months        671205 non-null int16
lender_count          671205 non-null int16
repayment_interval    671205 non-null category
country_alpha_3       671205 non-null category
share_gathered        671205 non-null float64
success               671205 non-null category
dtypes: category(7), float64(1), int16(2), int32(2), object(1)
memory usage: 23.1+ MB


In [24]:
mem2 = df.memory_usage(deep=True).sum()

In [25]:
reduction = (mem1 - mem2) / mem1 * 100
print(f"The memory usage has been reduced by {reduction:0.1f}%.")

The memory usage has been reduced by 80.3%.


### Analysis

The main question - why are some projects successful and others not? Will try to understand.

#### Our geography

With which countries do we work?

In [26]:
# Preparing a separate dataframe for the map
countries = df.groupby(['country', 'country_alpha_3']).agg({'funded_amount': ['size', 'sum', 'mean'], 'lender_count':'mean', 'share_gathered':'mean'})
countries.reset_index(inplace=True)
countries.columns = countries.columns.get_level_values(1)
countries.columns = ['Country', 'Country code', '# of projects', 'Total money raised, $', 'Avg money raised pro project, $', 'Avg investors', 'Avg share gathered, %']
countries = countries.dropna()

In [27]:
countries.head(3)

,Country,Country code,# of projects,"Total money raised, $","Avg money raised pro project, $",Avg investors,"Avg share gathered, %"
0,Afghanistan,AFG,2.0,14000.0,7000.000000,256.000000,100.000000
88,Albania,ALB,1934.0,2490000.0,1287.487073,42.904343,94.441943
176,Armenia,ARM,8631.0,11186675.0,1296.104159,37.140076,88.457472


In [28]:
fig = px.choropleth(countries, locations="Country code",
                    color="# of projects",
                    hover_name="Country",
                    projection='robinson',
                    title='From which countries do projects come?',
                    color_continuous_scale=px.colors.sequential.Plasma)
fig.show()

Coloured countries are those from where business ideas are coming to us. <br>
Many developed countries still do not take advantage of our service. Are we not known to entrepreneurs there? <br>
We have many requests / projects from Philippines (I wonder why). <br>
My suggestion: try to attract ventures from developed countries: EU, Japan, Australia, Canada.

#### How many of the pitched projects manage to raise funds?

In [29]:
# Defining a function for plotting donut charts:

def plotdonut(data, name, legend):
    success_pie = px.pie(data_frame=data,
                names=data.index,
                values=0,
                title=name,
                template='simple_white',
                labels={'0':'# of projects'},
                color_discrete_sequence=["#002b80", "#99c2ff", "#c2c2d6", "#ff6666"],                
                              )

    success_pie.update_traces(
                textinfo='percent', 
                hole=0.5, 
                texttemplate='%{percent:.0%f}',
                direction='clockwise',
                        )

    success_pie.update_layout(
                font_family="Verdana",
                font_color="darkblue",
                font={'size':16},
                title_font_family="Verdana",
                title={'x':0.35,
                       'y':0.96,
                       'xanchor':'center',
                       'yanchor':'top'},
                legend_title_text=legend,
                legend_traceorder='normal'
                     )

    success_pie.show()

In [30]:
# How many projects do we have, according to 'success'?
successful = df.groupby('success').size()
successful

success
unsuccessful           6614
partly successful     20106
mostly successful     20215
successful           624270
dtype: int64

In [31]:
# Plotting it as a donut-chart:
plotdonut(successful, 'Share of projects by success of crowdfunding', 'All projects:')

Most projects (93%) were successful in crowdfunding - our platform does its job well.

#### Analyzing Activities

In [32]:
# Average money gathered pro activity? Projects in which activities are most expensive ones, on average?

activities = df.groupby(['sector', 'activity']).agg({'funded_amount': ['size', 'sum', 'mean'], 'lender_count':'mean', 'share_gathered':'mean'})
activities.reset_index(inplace=True)
activities.columns = activities.columns.get_level_values(1)
activities.columns = ['Sector', 'Activity', '# of projects', 'Total money raised, $', 'Avg money raised pro project, $', 'Avg investors', 'Avg share gathered, %']
activities = activities.dropna()

The following parameter - 'Value over volume' - will reflect 
how many times the share of an activity in money (Value) is bigger or smaller 
than the share of an activity in the total number of projects (Volume).
So if 'Value over volume' for a particular activity is 2 
it means that projects related to this activity "weigh" 2 times more in money attracted than in their simple count.
In other words, the share of such projects in all money raised through our platform is 2 times bigger 
than their share in the total number of projects.

In [33]:
activities['Share in volume, %'] = activities['# of projects'] / activities['# of projects'].sum()
activities['Share in value, %'] = activities['Total money raised, $'] / activities['Total money raised, $'].sum()
activities['Value over volume'] = round(activities['Share in value, %'] / activities['Share in volume, %'], 1)

In [34]:
for c in ['Avg investors', 'Avg money raised pro project, $', 'Avg share gathered, %']:
    activities[c] = round(activities[c], 1)

In [35]:
activities.head(3)

,Sector,Activity,# of projects,"Total money raised, $","Avg money raised pro project, $",Avg investors,"Avg share gathered, %","Share in volume, %","Share in value, %",Value over volume
1,Agriculture,Agriculture,27023.0,25530735.0,944.8,26.9,95.3,0.040260,0.048394,1.2
3,Agriculture,Animal Sales,9237.0,9975500.0,1080.0,27.7,96.7,0.013762,0.018909,1.4
4,Agriculture,Aquaculture,108.0,71300.0,660.2,19.2,93.2,0.000161,0.000135,0.8


Which sectors and activities are represented in the projects?

In [36]:
fig = px.treemap(activities, 
                 path=['Sector', 'Activity'], 
                 values='# of projects', 
                 color='Avg money raised pro project, $',
                 hover_data=["Value over volume", "Avg investors"],
                 title='Sectors and Activities'
                )
fig.show()

This treemap shows 

a) shares of sectors and activities in the general volume of projects (% from the whole amount), 
represented by sizes of blocks,

b) average amount of money attracted by projects related to each specific activity, 
reflected by colour (the lighter / closer to yellow, the more expensive the associated projects are).

We see one small yellow block located in the Retail sector, standing out against the whole graph:
this is 'Renewable Energy Projects'. The bright colour hints that corresponding projects are on average much more expensive than most others, coloured in hues of purple.

If we click on the block 'Services' (as the segment) and magnify it, we can see some more light coloured blocks representing activities such as 'Landscaping / Gardening', 'Technology', 'Childcare', 'Communications', and 'Cleaning services' - they are also expensive but unfortunately appearing too seldom on our platform.

In [37]:
# Which projects attract more money, on average?
activities.sort_values(by='Avg money raised pro project, $', ascending=False).head(10)

,Sector,Activity,# of projects,"Total money raised, $","Avg money raised pro project, $",Avg investors,"Avg share gathered, %","Share in volume, %","Share in value, %",Value over volume
2042,Services,Landscaping / Gardening,34.0,123765.0,3640.1,74.2,81.1,0.000051,0.000235,4.6
1924,Retail,Renewable Energy Products,585.0,1798675.0,3074.7,83.4,99.6,0.000872,0.003409,3.9
2100,Services,Technology,178.0,428245.0,2405.9,44.2,47.0,0.000265,0.000812,3.1
1992,Services,Communications,96.0,223285.0,2325.9,42.1,62.4,0.000143,0.000423,3.0
1986,Services,Child Care,183.0,417300.0,2280.3,61.4,86.1,0.000273,0.000791,2.9
1987,Services,Cleaning Services,145.0,323085.0,2228.2,46.8,70.7,0.000216,0.000612,2.8
221,Arts,Film,13.0,27200.0,2092.3,62.7,100.0,0.000019,0.000052,2.7
478,Clothing,Used Shoes,752.0,1392400.0,1851.6,35.0,98.0,0.001120,0.002639,2.4
1854,Retail,Florist,65.0,118925.0,1829.6,39.9,92.1,0.000097,0.000225,2.3
1902,Retail,Perfumes,328.0,557725.0,1700.4,25.5,95.8,0.000489,0.001057,2.2


In [38]:
# More focussed:
activities.loc[activities['Value over volume'] >= 2, ['Activity', '# of projects', 'Value over volume', 'Avg share gathered, %']].sort_values(by='Value over volume', ascending=False)

,Activity,# of projects,Value over volume,"Avg share gathered, %"
2042,Landscaping / Gardening,34.0,4.6,81.1
1924,Renewable Energy Products,585.0,3.9,99.6
2100,Technology,178.0,3.1,47.0
1992,Communications,96.0,3.0,62.4
1986,Child Care,183.0,2.9,86.1
1987,Cleaning Services,145.0,2.8,70.7
221,Film,13.0,2.7,100.0
478,Used Shoes,752.0,2.4,98.0
1854,Florist,65.0,2.3,92.1
1902,Perfumes,328.0,2.2,95.8


There are certain activities which are seldom represented in projects we mediate 
but which attract much more money (in sum) than their simple numerical share in all projects would suggest.
In brief, they are expensive.<br>
Projects related to these 12 activities are all at least 2 times bigger in money/value than in volume.<br> 
Most of them manage to raise almost all needed money (with exception of Technology, Communications, and Cleaning Services)<br>
We need more such projects on our platform.

In [39]:
# Which projects attract more investors, on average?
activities.sort_values(by='Avg investors', ascending=False).head(10)

,Sector,Activity,# of projects,"Total money raised, $","Avg money raised pro project, $",Avg investors,"Avg share gathered, %","Share in volume, %","Share in value, %",Value over volume
1924,Retail,Renewable Energy Products,585.0,1798675.0,3074.7,83.4,99.6,0.000872,0.003409,3.9
2042,Services,Landscaping / Gardening,34.0,123765.0,3640.1,74.2,81.1,0.000051,0.000235,4.6
221,Arts,Film,13.0,27200.0,2092.3,62.7,100.0,0.000019,0.000052,2.7
1986,Services,Child Care,183.0,417300.0,2280.3,61.4,86.1,0.000273,0.000791,2.9
1956,Services,Adult Care,2.0,3375.0,1687.5,51.5,100.0,0.000003,0.000006,2.1
2011,Services,Event Planning,5.0,7175.0,1435.0,48.2,100.0,0.000007,0.000014,1.8
2355,Wholesale,Goods Distribution,277.0,462650.0,1670.2,48.1,98.4,0.000413,0.000877,2.1
1987,Services,Cleaning Services,145.0,323085.0,2228.2,46.8,70.7,0.000216,0.000612,2.8
2100,Services,Technology,178.0,428245.0,2405.9,44.2,47.0,0.000265,0.000812,3.1
1217,Health,Health,1235.0,1799800.0,1457.3,44.2,97.1,0.001840,0.003412,1.9


These are again projects related to Landscaping / Gardening and Renewable Energy Products.
Renewable Energy Products is especially remarkable because money is almost always bein gathered in full ('Avg share gathered' ~ 100%) despite expensiveness AND Renewable Energy Products are leading by the number of investors attracted on average. Seems to be a hot topic! At the same time, they are relatively seldom pitched, in comparison to most other themes / activities.

##### My suggestion is to 

##### a) focus on ventures which are initiating projects related to Renewable Energy Products and try to make them using our service, 

##### b) try to attract also ventures doing projects connected to activities with Value over volume > 2,

##### c) maybe change our commissioning policy and take higher commission for projects with Value over volume > 2.

In [40]:
fig = go.Figure(
    px.scatter(activities, 
           x="Avg money raised pro project, $", 
           y="# of projects", 
           size='Avg investors', 
           color='Sector',
           hover_name='Activity',
           opacity=0.6,
           #log_y=True,
           title="Activities: projects' prevalence vs. expensiveness",
           hover_data=['Value over volume', 'Avg share gathered, %'],
           #template="plotly_white",
          )
)
fig.show()

All activities in the central and the right parts of the graph (starting approx. from 1500 $ on x-axis) tend to attract much money and many investors; we need to make them appear on our platform more often - here I see potential for good profits!

In [41]:
# And which projects have problems with fundrising?
activities.sort_values(by='Avg share gathered, %', ascending=True).head(5)

,Sector,Activity,# of projects,"Total money raised, $","Avg money raised pro project, $",Avg investors,"Avg share gathered, %","Share in volume, %","Share in value, %",Value over volume
2100,Services,Technology,178.0,428245.0,2405.9,44.2,47.0,0.000265,0.000812,3.1
1992,Services,Communications,96.0,223285.0,2325.9,42.1,62.4,0.000143,0.000423,3.0
1987,Services,Cleaning Services,145.0,323085.0,2228.2,46.8,70.7,0.000216,0.000612,2.8
869,Entertainment,Entertainment,443.0,643920.0,1453.5,33.9,79.8,0.000660,0.001221,1.8
2042,Services,Landscaping / Gardening,34.0,123765.0,3640.1,74.2,81.1,0.000051,0.000235,4.6


These are 'Technology', 'Communications', 'Cleaning Services' projects. They are actually quite expensive (see Value over volume)...
What are their problems? From which countries do they come?

#### Countries with successful / expensive projects

What are the leaders in terms of total money funded?

In [42]:
fig = px.choropleth(countries, locations="Country code",
                    color='Total money raised, $',
                    hover_name="Country",
                    projection='robinson',
                    title='Which countries attract more money, in total?', 
                    color_continuous_scale=px.colors.sequential.Plasma)
fig.show()

Big money is flowing to Phillipines, Kenya, Peru, Paraguay, USA.

And what is average money raised pro project, by country?
I will sort out countries where there were less than 10 projects so that they don't make noise with their 1-2 projects.

In [43]:
data = countries.loc[countries['# of projects']>=10]

fig = px.choropleth(data, locations="Country code",
                    color='Avg money raised pro project, $',
                    hover_name="Country",
                    hover_data=["# of projects"],
                    title='Average sum raised pro project, $', 
                    color_continuous_scale=px.colors.sequential.Plasma,
                    projection='robinson'
    )
fig.show()

Chile has the most expensive projects, followed by Congo.

Projects related to 12 lucrative activities - where do they come from?

In [44]:
data = df.loc[df['activity'].isin([
    'Landscaping / Gardening',
    'Renewable Energy Products',
    'Technology',
    'Communications',
    'Child Care',
    'Cleaning Services',
    'Film',
    'Used Shoes',
    'Florist',
    'Perfumes',
    'Adult Care',
    'Goods Distribution'
])
                    ]

In [45]:
countries_sel = data.groupby(['country', 'country_alpha_3']).agg({'funded_amount': ['size', 'sum', 'mean'], 'lender_count':'mean', 'share_gathered':'mean'})
countries_sel.reset_index(inplace=True)
countries_sel.columns = countries_sel.columns.get_level_values(1)
countries_sel.columns = ['Country', 'Country code', '# of projects', 'Total money raised, $', 'Avg money raised pro project, $', 'Avg investors', 'Avg share gathered, %']
countries_sel = countries_sel.dropna()

In [46]:
fig = px.choropleth(countries_sel, locations="Country code",
                    color='# of projects',
                    hover_name="Country",
                    title="Applications for particularly expensive projects - where do they come from?", 
                    color_continuous_scale=px.colors.sequential.Plasma,
                    projection='robinson'
    )
fig.show()

Most pitches from 12 'lucrative' categories are coming from USA. 
How successful are they and what is the structure of themes / activities?

In [47]:
usa_sel = data.loc[data['country'] == 'United States']

In [48]:
successful_usa_sel = usa_sel.groupby('success').size()
successful_usa_sel

success
unsuccessful         165
partly successful     51
mostly successful     20
successful           279
dtype: int64

In [49]:
# Plot it as a pie-chart:
plotdonut(successful_usa_sel, 'US-Projects in the key spheres by success of crowdfunding', 'Big value projects:')

In [50]:
usa = df.loc[df['country'] == 'United States']

In [51]:
usa.head(4)

,funded_amount,loan_amount,activity,sector,use,country_code,country,term_in_months,lender_count,repayment_interval,country_alpha_3,share_gathered,success
99,5000,5000,Food Production/Sales,Food,hire more staff to keep up with the volume of ...,US,United States,30,261,monthly,USA,100.0,successful
116,4000,4000,Cosmetics Sales,Retail,to purchase more inventory to sell,US,United States,26,93,monthly,USA,100.0,successful
231,10000,10000,Food Production/Sales,Food,outfit my new kitchen so that I can expand my ...,US,United States,36,280,monthly,USA,100.0,successful
455,5000,5000,Farming,Agriculture,to purchase a tractor,US,United States,32,158,monthly,USA,100.0,successful


In [52]:
successful_usa = usa.groupby('success').size()
successful_usa

success
unsuccessful         1409
partly successful     463
mostly successful     288
successful           3933
dtype: int64

In [53]:
# Plot it as a pie-chart:
plotdonut(successful_usa, 'All US-projects by success of crowdfunding', 'All projects:')

In [54]:
# And now comparing the distributions:

import plotly.graph_objects as go
from plotly.subplots import make_subplots

labels = ["unsuccesful", "partly succesful", "mostly succesful", "successful"]
specs = [[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}]]
colors = ["#ff6666", "#c2c2d6", "#99c2ff", "#002b80"]


fig = make_subplots(rows=1, cols=3, subplot_titles=("World", "USA all", "USA key spheres"), specs=specs)

fig.add_trace(go.Pie(
                labels=labels, 
                values=successful.values,
                name="World",
                pull=[0.05, 0, 0, 0],
                                   ), 1, 1
                              )
    
fig.add_trace(go.Pie(
                labels=labels, 
                values=successful_usa.values,
                name="USA all",
                pull=[0.05, 0, 0, 0],
                                   ), 1, 2
                              )
    
fig.add_trace(go.Pie(
                labels=labels, 
                values=successful_usa_sel.values,
                name="USA key spheres",
                pull=[0.05, 0, 0, 0],
                                   ), 1, 3
                              )
    
fig.update_traces(
                textinfo='percent', 
                hole=0.5, 
                texttemplate='%{percent:.0%f}',
                marker_colors=colors,
                        )

fig = go.Figure(fig)

fig.show()

The USA do indeed differ markedly from the total picture: much more projects (almost 1/4 in general and 1/3 in the most attractive markets) from the States fail to raise money.<br>
Reasons? Will attempt to explore them below.

Projects related specifically to Renewable Energy Products - where do they come from?

In [55]:
rep = df.loc[df["activity"] == 'Renewable Energy Products']

renewable = rep.groupby(['country', 'country_alpha_3']).agg({'funded_amount': ['size', 'sum', 'mean'], 'lender_count':'mean', 'share_gathered':'mean'})
renewable.reset_index(inplace=True)
renewable.columns = renewable.columns.get_level_values(1)
renewable.columns = ['Country', 'Country code', '# of projects', 'Total money raised, $', 'Avg money raised pro project, $', 'Avg investors', 'Avg share gathered, %']
renewable = renewable.dropna()

In [56]:
renewable.loc[:, '# of projects'].sum()

585.0

In [57]:
fig = px.choropleth(renewable, locations="Country code",
                    color='# of projects',
                    hover_name="Country",
                    hover_data=['Avg money raised pro project, $', 'Avg share gathered, %'],
                    title='Renewable Energy Products by origin', 
                    color_continuous_scale=px.colors.sequential.Plasma,
                    projection='robinson'
    )
fig.show()

Out of 585 projects, 186 are in Kenya, 123 in Liberia, 89 on Haiti. 
Why are there so many initiatives coming from these countries? I would look in depth but need more information:
Who are the the clients? Do their businesses really operate in these countries or they are only registered there?
Is there anything happening in particular, for example, in connection to govermental policy which is promoting
Renewable Energy? Should we expect more such pitches?
And - any similar tendencies in other countries? Why the projects or at least pitches on our platform so geographically concentrated?

##### Potentially, we can grow an entirely separate branch of business related to projects in Renewable Energy only, leaving our current platform for everything else.

#### Understanding unsuccessful projects: any visible tendencies?

Projects from which countries are on average less successful in gathering funds?

In [58]:
fig = px.choropleth(countries, locations="Country code",
                    color='Avg share gathered, %',
                    hover_name="Country",
                    hover_data=["# of projects"],
                    title='Effectiveness of fundrising', 
                    color_continuous_scale=px.colors.sequential.Plasma,
                    projection='robinson'
    )
fig.show()

Definitely USA: only 70% are raised on average. Which activities are so unsuccessful in USA?

In [59]:
def success_pivot(variable):
    table = pd.pivot_table(usa, values='sector', index=variable, columns='success', aggfunc='count', margins=True)
    table = table.loc[table['All'] >= 10]
    table['uns_row_%'] = round((table['unsuccessful']/(table['All']))*100, 1)
    table['ps_row_%'] = round((table['partly successful']/(table['All']))*100, 1)
    table['ms_row_%'] = round((table['mostly successful']/(table['All']))*100, 1)
    table['succ_row_%'] = round((table['successful']/(table['All']))*100, 1)
    return table.sort_values(by='succ_row_%', ascending=True).head(10)

In [60]:
# USA - how activities are distributed by success of crowdfunding? (Only where there were 10 or more pitches.)
success_pivot('activity')

success,unsuccessful,partly successful,mostly successful,successful,All,uns_row_%,ps_row_%,ms_row_%,succ_row_%
activity,,,,,,,,,
Technology,72.0,26.0,10.0,64.0,172,41.9,15.1,5.8,37.2
Entertainment,77.0,14.0,4.0,79.0,174,44.3,8.0,2.3,45.4
Property,13.0,2.0,3.0,15.0,33,39.4,6.1,9.1,45.5
Retail,51.0,24.0,18.0,80.0,173,29.5,13.9,10.4,46.2
Grocery Store,7.0,7.0,1.0,15.0,30,23.3,23.3,3.3,50.0
Communications,29.0,8.0,4.0,45.0,86,33.7,9.3,4.7,52.3
Home Appliances,NaN,2.0,4.0,7.0,13,NaN,15.4,30.8,53.8
Services,371.0,124.0,75.0,753.0,1323,28.0,9.4,5.7,56.9
Transportation,37.0,6.0,7.0,68.0,118,31.4,5.1,5.9,57.6


Projects related to the following activities fail in the USA most often: Technology, followed by Entertainment, Property, Retail. Remarkable are also projects related to Services because of their big volume.

Does this dataset provide any evidence about important differences between successful and unsuccessful projects?
##### I am focusing on the USA.
I won't usa ANOVA or other tests, will only look at percentages and try to see tendencies.

Mean, min, max 'loan_amount'? 
Differences in means (min, max) of 'term_in_months' or 'lender_count'?
Anything to do with 'repayment_interval'?

In [61]:
def descr_table(variable):    
    table = usa.groupby('success').agg({variable: ['size', 'min', 'mean', 'median', 'max']})
    return table

In [62]:
# Differences in means (min, max) of 'lender_count'
descr_table('lender_count')

lender_count                            
                          size min        mean median  max
success                                                   
unsuccessful              1409   0    1.935415      0   55
partly successful          463   1   36.146868     34  187
mostly successful          288   1   72.357639     64  268
successful                3933   1  100.178998     87  507

As expected, the number of investors is strongly correlated to the share of money raised. No suprise.
But why are they attracted to some projects and not to others? Because of their spheres / topics / purposes?..

In [63]:
# Mean, min, max 'loan_amount'
descr_table('loan_amount')

loan_amount                                
                         size  min         mean median    max
success                                                      
unsuccessful             1409  125  4262.313698   4975  10000
partly successful         463  500  5665.010799   5000  10000
mostly successful         288  500  5741.145833   5000  10000
successful               3933  200  5393.954996   5000  10000

No big difference between successful and unsuccessful projects.
Unsuccessful ones claim even lower loan amount (if we take means) but I don't think this parameter matters, at least on its own, without interplay with something else.

In [64]:
# Differences in means (min, max) of 'term_in_months'?
descr_table('term_in_months')
# I don't see any tendency here as well.

term_in_months                          
                            size min       mean median max
success                                                   
unsuccessful                1409   6  24.129879     24  63
partly successful            463  12  27.645788     24  63
mostly successful            288   6  29.482639     26  63
successful                  3933   3  27.751589     24  63

In [65]:
# Anything to do with 'repayment_interval'?
success_pivot('repayment_interval')

success,unsuccessful,partly successful,mostly successful,successful,All,uns_row_%,ps_row_%,ms_row_%,succ_row_%
repayment_interval,,,,,,,,,
bullet,1405.0,388.0,156.0,46.0,1995,70.4,19.4,7.8,2.3
All,1409.0,463.0,288.0,3933.0,6093,23.1,7.6,4.7,64.5
monthly,4.0,75.0,132.0,3869.0,4080,0.1,1.8,3.2,94.8
irregular,NaN,NaN,NaN,18.0,18,NaN,NaN,NaN,100.0


Unsuccessful projects are more often associated with 'bullet' payment mode. Does it mean that enterprisers just pay this small money (if they got any) immediately after they realize that they won't succeed? Can be.
By the way, weekly interval doesn't appear in the USA at all (but it is seldom anyway). 
Otherwise I cannot make any reasonable conclusion. I believe it has to do something rather with the topic (and quality of its presentation).

From this database alone, I don't understand why some projects in the USA are more successful than others. We know that projects related to Technology, Entertainment, Property, Retail, Services fail in the USA most often.

Are these projects (or generally projects related to more or less all activities in the USA) pitched by the entrepreneurs with much different quality? Is it just too many ideas coming from USA, - many of them not described and pitched well enough, - simply because of especially high value of entrepreneurship in this country?

Do they demand more money than investors believe reasonable? Or do investors for some reasons show less interest for the abovementioned topics?

##### These are questions for a separate study. Maybe we could change our marketing / information strategy in the USA, making it more clear to potential users how projects should be presented, to look attractive enough for investors.

### Summary

##### 1. Most projects (93%) were successful in crowdfunding - our platform does its job well.
##### 2. Ventures from developed countries (EU, Japan, Australia, Canada) do not use our service or don't know us: we need to approach them.
##### 3. The biggest share in the total volume of projects we manage belongs to projects related to 'Farming', 'General Store', 'Agriculture', 'Housing', 'Clothing'.
##### 4. The most lucrative projects which attract on average much more money than others and bring the highest provision / profit belong however to other spheres: ‘Landscaping / Gardening’, ‘Renewable Energy Products’, ‘Technology’, ‘Communications’, ‘Child Care’, ‘Cleaning Services’’. However, unfortunately, they are very seldom pitched on our platform, compared to projects in many other activities. We need more such projects on our platform and should try to attract ventures doing projects connected to these activities.
##### 5. We can consider increasing commission on these projects: the typically high number of interested investors and usual success in crowdfunding both suggest that slightly increased commission should not be a barrier to entrepreneurs or investors.
##### 6. Our focus number one should be projects related to Renewable Energy Products: they are popular among investors and are able to raise big money. We need to find ventures which are initiating projects related to Renewable Energy Products and try to make them using our service.
##### 7. Too many projects from USA are failing to raise enough or even little money. Further research is needed on reasons for it, in order to find ways to help American entrepreneurs find investors which would help us achieve more sound results on the US market.